In [84]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
import warnings; warnings.simplefilter('ignore')
from scipy import sparse

In [15]:
data = pd. read_csv('movies_data.csv')
data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,30-10-1995,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,15-12-1995,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,22-12-1995,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,22-12-1995,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,10-02-1995,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173


In [16]:
ratings= pd.read_csv("ratings.csv")

In [17]:
ratings= ratings[['userId', 'movieId', 'rating']]
ratings=ratings.head(1000000)

In [18]:
#renaming the column
data= data[['id', 'original_title', 'original_language']]
data= data.rename(columns={'id':'movieId'})
data = data[data['original_language']== 'en'] 
data

,movieId,original_title,original_language
0,862,Toy Story,en
1,8844,Jumanji,en
2,15602,Grumpier Old Men,en
3,31357,Waiting to Exhale,en
4,11862,Father of the Bride Part II,en
...,...,...,...
2594,10208,Muppets from Space,en
2595,2667,The Blair Witch Project,en
2596,48958,My Life So Far,en
2597,345,Eyes Wide Shut,en


In [19]:
#Merging two dataset(Movies_data and ratings)
data.movieId =pd.to_numeric(data.movieId, errors='coerce')
ratings.movieId = pd.to_numeric(ratings.movieId, errors= 'coerce')
meta= pd.merge(ratings, data, on='movieId', how='inner')
meta.head()

,userId,movieId,rating,original_title,original_language
0,1,1371,2.5,Rocky III,en
1,4,1371,4.0,Rocky III,en
2,7,1371,3.0,Rocky III,en
3,19,1371,4.0,Rocky III,en
4,21,1371,3.0,Rocky III,en


In [20]:
matrix= meta.pivot_table(index='userId', columns='original_title', values='rating')
matrix.head()

original_title,10 Things I Hate About You,12 Angry Men,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2010,3 Ninjas: High Noon at Mega Mountain,54,A Christmas Carol,A Clockwork Orange,A Close Shave,...,When Harry Met Sally...,While You Were Sleeping,Wild Things,Wild Wild West,Willy Wonka & the Chocolate Factory,Working Girl,Young Frankenstein,Young and Innocent,eXistenZ,¡Three Amigos!
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN
4,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN


In [21]:
#defining pearson function
def pearsonR(s1, s2):
    s1_c = s1-s1.mean()
    s2_c= s2-s2.mean()
    return np.sum(s1_c*s2_c) / np.sqrt(np.sum(s1_c**2)* np.sum(s2_c**2))

def recommend(movie, M, n):
    reviews=[]
    for title in M.columns:
        if title == movie:
            continue
        cor= pearsonR(M[movie], M[title])
        if np.isnan(cor):
            continue
        else:
            reviews.append((title, cor))
            
    reviews.sort(key= lambda tup: tup[1], reverse=True)
    return reviews[:n]

In [22]:
#watched movie list
watched = ['Godzilla', 'Die Hard', 'Donnie Darko', 'Waiting to Exhale', 'Men in Black II' ]

In [23]:
#top 20 recommendations
recs = recommend('The Man Who Knew Too Much', matrix, 20) 
recs

[('Die Hard', 0.605055236564425),
 ('Godzilla', 0.4140393356054125),
 ('Bushwhacked', 0.39694209301872235),
 ('Midnight in the Garden of Good and Evil', 0.37465377150775275),
 ('Cabin Boy', 0.3273268353539886),
 ('The River Wild', 0.3273268353539886),
 ('Wild Wild West', 0.3273268353539886),
 ('Michael', 0.3149703941743559),
 ('Amadeus', 0.31492193346260305),
 ("What's Eating Gilbert Grape", 0.3022269466299638),
 ('Welcome to Woop Woop', 0.29277002188455997),
 ('54', 0.29115695163718175),
 ('Casablanca', 0.26966354240402013),
 ('Under Siege 2: Dark Territory', 0.2674013742718877),
 ("My Best Friend's Wedding", 0.2557712322055721),
 ('Sabrina', 0.2433145233315402),
 ('Eyes Wide Shut', 0.22680552628462727),
 ('Spellbound', 0.2213133340689952),
 ('Casino', 0.21941203889890354),
 ('Marnie', 0.2185207604584122)]

In [24]:
#trimmed movie list
trimmed_recommendation_list = [r for r in recs if r[0] not in watched]
trimmed_recommendation_list

[('Bushwhacked', 0.39694209301872235),
 ('Midnight in the Garden of Good and Evil', 0.37465377150775275),
 ('Cabin Boy', 0.3273268353539886),
 ('The River Wild', 0.3273268353539886),
 ('Wild Wild West', 0.3273268353539886),
 ('Michael', 0.3149703941743559),
 ('Amadeus', 0.31492193346260305),
 ("What's Eating Gilbert Grape", 0.3022269466299638),
 ('Welcome to Woop Woop', 0.29277002188455997),
 ('54', 0.29115695163718175),
 ('Casablanca', 0.26966354240402013),
 ('Under Siege 2: Dark Territory', 0.2674013742718877),
 ("My Best Friend's Wedding", 0.2557712322055721),
 ('Sabrina', 0.2433145233315402),
 ('Eyes Wide Shut', 0.22680552628462727),
 ('Spellbound', 0.2213133340689952),
 ('Casino', 0.21941203889890354),
 ('Marnie', 0.2185207604584122)]

In [123]:
#Simple recommender
md = pd. read_csv('movies_data.csv')
md

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,30-10-1995,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,15-12-1995,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,22-12-1995,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,22-12-1995,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,10-02-1995,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2594,False,"{'id': 256377, 'name': 'The Muppet Collection'...",24000000,"[{'id': 14, 'name': 'Fantasy'}, {'id': 35, 'na...",NaN,10208,tt0158811,en,Muppets from Space,When Gonzo's breakfast cereal tells him that h...,...,14-07-1999,16290976,87.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Space. It's not as deep as you think.,Muppets from Space,False,5.8,94
2595,False,"{'id': 64750, 'name': 'Blair Witch Collection'...",60000,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",http://www.blairwitch.com/,2667,tt0185937,en,The Blair Witch Project,In October of 1994 three student filmmakers di...,...,14-07-1999,248000000,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The scariest movie of all time is a true story.,The Blair Witch Project,False,6.3,1090
2596,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,48958,tt0120899,en,My Life So Far,Memoir of the lives of a family growing up on ...,...,25-08-1999,0,98.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Imagine a summer in paradise with nothing to d...,My Life So Far,False,6.1,10
2597,False,NaN,65000000,"[{'id': 9648, 'name': 'Mystery'}, {'id': 18, '...",http://eyeswideshut.warnerbros.com/,345,tt0120663,en,Eyes Wide Shut,"After Dr. Bill Hartford's wife, Alice, admits ...",...,14-07-1999,162091208,159.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Cruise. Kidman. Kubrick.,Eyes Wide Shut,False,7.1,1266


In [124]:
#Weighted rating formula (Weighted Rating (WR) =  (vv+m.R)+(mv+m.C))
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [125]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.656021546748749

In [126]:
#Counting the number of movies which have more votes than at least 95% of the movies in the list.
m = vote_counts.quantile(0.85)
m

483.2999999999997

In [127]:
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [128]:
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(390, 6)

In [129]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [130]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [131]:
qualified = qualified.sort_values('wr', ascending=False).head(250)

In [132]:
#Top 15 movies
qualified.head(15)

,title,year,vote_count,vote_average,popularity,genres,wr
292,Pulp Fiction,1994,8670,8,140.950236,"[Thriller, Crime]",7.876236
314,The Shawshank Redemption,1994,8358,8,51.645403,"[Drama, Crime]",7.871869
351,Forrest Gump,1994,8147,8,48.307194,"[Comedy, Drama, Romance]",7.868736
256,Star Wars,1977,6778,8,42.149697,"[Adventure, Action, Science Fiction]",7.843989
1225,Back to the Future,1985,6239,8,25.778509,"[Adventure, Comedy, Science Fiction, Family]",7.831480
834,The Godfather,1972,6024,8,41.109264,"[Drama, Crime]",7.825912
1154,The Empire Strikes Back,1980,5998,8,19.470959,"[Adventure, Action, Science Fiction]",7.825213
46,Se7en,1995,5915,8,18.457430,"[Crime, Mystery, Thriller]",7.822946
359,The Lion King,1994,5520,8,21.605761,"[Family, Animation, Drama]",7.811296
586,The Silence of the Lambs,1991,4549,8,4.307222,"[Crime, Drama, Thriller]",7.774885


In [133]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)

In [134]:
#defining function to display the chart based on the genre
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

In [135]:
#Top 15 Romantic movies
build_chart('Romance').head(15)

,title,year,vote_count,vote_average,popularity,wr
351,Forrest Gump,1994,8147,8,48.307194,7.918515
876,Vertigo,1958,1162,8,18.208220,7.531992
883,Some Like It Hot,1959,835,8,11.845107,7.398397
1132,Cinema Paradiso,1988,834,8,14.177005,7.397872
882,The Apartment,1960,498,8,11.994281,7.147652
1639,Titanic,1997,7770,7,26.889070,6.953145
2178,Edward Scissorhands,1990,3731,7,17.612244,6.906319
581,Aladdin,1992,3495,7,16.357419,6.900510
588,Beauty and the Beast,1991,3029,7,23.433511,6.886627
1220,Groundhog Day,1993,2358,7,12.989627,6.858119


In [136]:
#Top 15 Action movies
build_chart('Action').head(15)

,title,year,vote_count,vote_average,popularity,wr
256,Star Wars,1977,6778,8,42.149697,7.714323
1154,The Empire Strikes Back,1980,5998,8,19.470959,7.682008
2458,The Matrix,1999,9079,7,33.366332,6.870165
1167,Return of the Jedi,1983,4763,7,14.586087,6.771168
1171,Alien,1979,4564,7,23.377420,6.762829
582,Terminator 2: Judgment Day,1991,4274,7,22.661695,6.749529
1195,The Terminator,1984,4208,7,19.293562,6.746292
1007,Die Hard,1988,4005,7,16.640522,6.735786
1460,The Fifth Element,1997,3962,7,24.305260,6.733448
1156,Raiders of the Lost Ark,1981,3949,7,19.901576,6.732733


In [137]:
#Top 15 Animated Movies
build_chart('Animation').head(15)

,title,year,vote_count,vote_average,popularity,wr
359,The Lion King,1994,5520,8,21.605761,7.599546
0,Toy Story,1995,5415,7,21.946943,6.835018
581,Aladdin,1992,3495,7,16.357419,6.774309
588,Beauty and the Beast,1991,3029,7,23.433511,6.752176
546,The Nightmare Before Christmas,1993,2135,7,17.730913,6.694746
1798,Mulan,1998,2089,7,21.180857,6.691062
1972,The Little Mermaid,1989,1921,7,17.496832,6.676818
1969,The Jungle Book,1967,1791,7,22.845448,6.664861
1495,Hercules,1997,1741,7,14.048694,6.660024
993,Cinderella,1950,1760,6,14.436858,6.156538


In [138]:
#Top 15 Comedy Movies
build_chart('Comedy').head(15)

,title,year,vote_count,vote_average,popularity,wr
351,Forrest Gump,1994,8147,8,48.307194,7.891655
1225,Back to the Future,1985,6239,8,25.778509,7.860445
2211,Life Is Beautiful,1997,3643,8,39.394970,7.770495
732,Dr. Strangelove or: How I Learned to Stop Worr...,1964,1472,8,9.803980,7.502126
883,Some Like It Hot,1959,835,8,11.845107,7.242085
1236,The Great Dictator,1940,756,8,9.241748,7.189590
882,The Apartment,1960,498,8,11.994281,6.952693
0,Toy Story,1995,5415,7,21.946943,6.906933
1604,The Truman Show,1998,4702,7,13.155901,6.893887
1902,Back to the Future Part II,1989,3926,7,13.689855,6.874785


In [139]:
#Metadata based recommender
credits = pd.read_csv('credits.csv')
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [140]:
keywords = pd.read_csv('keywords.csv')
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [141]:
#To have cast, crew, genres and credits, all in one dataframe
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [142]:
md.shape

(2599, 25)

In [143]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [144]:
links_small = pd.read_csv('link.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [145]:
smd = md[md['id'].isin(links_small)]
smd.shape

(2180, 28)

In [146]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [147]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [148]:
smd['director'] = smd['crew'].apply(get_director)

In [149]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [150]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x, x])

In [151]:
#calculating the frequenct counts of every keyword that appears in the dataset.
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [152]:
s = s.value_counts()
s[:5]

independent film    179
woman director      133
murder              103
based on novel       83
suspense             72
Name: keyword, dtype: int64

In [153]:
s = s[s > 1]
#converting every word to its stem
stemmer = SnowballStemmer('english')
stemmer.stem('cats')

'cat'

In [154]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [155]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [156]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [157]:
#Using CountVectorizer to create count matrix
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [158]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [159]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [160]:
#Function that returns the 30 most similar movies based on the cosine similarity score
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [161]:
get_recommendations('Toy Story').head(10)

1875                      A Bug's Life
1424                  Meet the Deedles
844                      Pete's Dragon
607                   Oliver & Company
1824                              Antz
320                    The Flintstones
845           Bedknobs and Broomsticks
1654    One Hundred and One Dalmatians
543                     The Aristocats
925                 The Wrong Trousers
Name: title, dtype: object

In [162]:
get_recommendations('The Godfather').head(10)

986      The Godfather: Part II
1338              The Rainmaker
1594    The Godfather: Part III
1683              The Outsiders
1984      Peggy Sue Got Married
642                        Jack
973              Apocalypse Now
1092                    Dracula
1757          The Paradine Case
860         Looking for Richard
Name: title, dtype: object

In [163]:
get_recommendations('Jumanji').head(10)

1624              Honey, I Shrunk the Kids
500                         The Pagemaster
1663                         The Rocketeer
2012                           October Sky
567              James and the Giant Peach
1613    Darby O'Gill and the Little People
1660            Return from Witch Mountain
451                                  North
552                All Dogs Go to Heaven 2
823               Escape to Witch Mountain
Name: title, dtype: object

In [164]:
#Improved recommendation 
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [165]:
improved_recommendations('The Godfather')

,title,vote_count,vote_average,year,wr
284,The Shawshank Redemption,8358,8,1994,7.871869
986,The Godfather: Part II,3418,8,1974,7.709624
978,GoodFellas,3211,8,1990,7.693353
973,Apocalypse Now,2112,8,1979,7.563501
1594,The Godfather: Part III,1589,7,1990,6.686559
1516,On the Waterfront,368,8,1954,6.669277
15,Casino,1343,7,1995,6.644338
1196,Donnie Brasco,1175,7,1997,6.608307
1092,Dracula,1087,7,1992,6.586356
1772,Rounders,451,6,1998,5.822065


In [183]:
#Content Based Recommender
md['id'] = md['id'].astype('int')
smd = md[md['id'].isin(links_small)]
smd.shape

(2180, 28)

In [184]:
#Description based
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [185]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])
tfidf_matrix.shape

(2180, 77375)

In [186]:
#Cosine similarity (cosine(x,y)=x.y⊺||x||.||y||)
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim[0]

array([1.        , 0.00823633, 0.        , ..., 0.        , 0.        ,
       0.00532227])

In [187]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [188]:
#using previously defined function to get top movie list
get_recommendations('The Godfather').head(10)

986      The Godfather: Part II
29               Shanghai Triad
625                     Thinner
2171              Summer of Sam
1594    The Godfather: Part III
227              The Jerky Boys
1554               Halloween II
1693          The Addams Family
818             The Glimmer Man
1396    The Replacement Killers
Name: title, dtype: object

In [189]:
get_recommendations('Toy Story').head(10)

902     Rebel Without a Cause
1611                Condorman
402         For Love or Money
1007                Manhattan
437                    Malice
1804        Indecent Proposal
1950                  Stepmom
878                   Sleeper
643        I Shot Andy Warhol
1564           Child's Play 3
Name: title, dtype: object

In [190]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies_data.csv')

In [191]:
#Merging two dataset
xy= pd.merge(ratings, data, on='movieId', how='inner').drop(['timestamp'],axis=1)
xy

,userId,movieId,rating,original_title,original_language
0,1,1371,2.5,Rocky III,en
1,4,1371,4.0,Rocky III,en
2,7,1371,3.0,Rocky III,en
3,19,1371,4.0,Rocky III,en
4,21,1371,3.0,Rocky III,en
...,...,...,...,...,...
10499,624,4477,1.0,The Devil's Own,en
10500,624,8447,1.0,Angels and Insects,en
10501,624,31000,2.5,Amos & Andrew,en
10502,641,638,4.0,Lost Highway,en


In [192]:
print(xy.shape)
xy.head()

(10504, 5)


,userId,movieId,rating,original_title,original_language
0,1,1371,2.5,Rocky III,en
1,4,1371,4.0,Rocky III,en
2,7,1371,3.0,Rocky III,en
3,19,1371,4.0,Rocky III,en
4,21,1371,3.0,Rocky III,en


In [193]:
#User_Rating
userRatings = xy.pivot_table(index=['userId'],columns=['original_title'],values='rating')
userRatings.head()
print("Before: ",userRatings.shape)
userRatings = userRatings.dropna(thresh=10, axis=1).fillna(0,axis=1)
print("After: ",userRatings.shape)

Before:  (657, 470)
After:  (657, 201)


In [194]:
#Using pearson method 
corrMatrix = userRatings.corr(method='pearson')
corrMatrix.head(100)

original_title,"20,000 Leagues Under the Sea",2001: A Space Odyssey,A Clockwork Orange,A Close Shave,A Nightmare on Elm Street,A Perfect Murder,A Time to Kill,A View to a Kill,Addicted to Love,Alien,...,Under Siege 2: Dark Territory,Vertigo,Wag the Dog,We're No Angels,What's Eating Gilbert Grape,While You Were Sleeping,Willy Wonka & the Chocolate Factory,Young Frankenstein,Young and Innocent,eXistenZ
original_title,,,,,,,,,,,,,,,,,,,,,
"20,000 Leagues Under the Sea",1.000000,0.041279,0.300758,0.093801,0.266908,0.054765,0.097291,0.041532,0.025775,0.064490,...,-0.052505,0.160687,0.157464,0.008384,0.209313,-0.015393,0.157722,0.153014,0.007826,-0.042969
2001: A Space Odyssey,0.041279,1.000000,0.177406,0.054005,0.167565,0.034318,0.054041,0.263699,0.068739,0.028214,...,0.050120,0.011937,0.186826,-0.010568,0.073213,-0.017806,0.237589,0.047658,0.015106,0.034785
A Clockwork Orange,0.300758,0.177406,1.000000,0.046780,0.411695,-0.004108,0.121483,0.045910,0.196815,0.003537,...,0.095593,0.067210,0.250399,-0.057767,0.049135,0.007013,0.301766,0.080059,0.045829,-0.023486
A Close Shave,0.093801,0.054005,0.046780,1.000000,0.108825,0.223483,0.100599,0.238245,0.030947,0.221743,...,-0.000046,0.250988,0.102875,0.019762,0.161604,0.188045,0.290695,0.077542,0.077653,0.124067
A Nightmare on Elm Street,0.266908,0.167565,0.411695,0.108825,1.000000,0.111793,0.179398,0.060545,0.235930,0.100964,...,0.114923,0.120350,0.373437,-0.034325,0.118281,0.083545,0.287159,0.144525,0.209630,0.099314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Married to the Mob,0.089236,0.189422,0.153072,0.169899,0.202874,0.208890,0.224687,0.214783,0.191459,0.219045,...,0.173965,0.051910,0.269524,-0.026798,0.191955,0.207886,0.204516,0.217425,0.262659,0.303285
Meet Me in St. Louis,-0.045675,0.075406,0.001560,0.096274,0.036956,0.143751,0.041685,0.057261,0.047754,0.185202,...,0.127166,0.075271,0.053921,-0.011070,0.055309,0.309493,0.118258,0.020790,0.080844,0.395838
Metro,0.030741,-0.007810,0.097249,-0.018853,0.147456,-0.025504,0.032968,0.015622,0.187766,0.001241,...,0.020801,-0.015365,0.123688,-0.023296,-0.010465,0.046456,0.003498,0.116436,0.125036,0.035317


In [195]:
#Top Movies based on the user ratings
def get_similar(movie_name,rating):
    similar_ratings = corrMatrix[movie_name]*(rating-2.5)
    similar_ratings = similar_ratings.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_ratings

In [196]:
action_movie_lover = [("The Terminator",4),("A Perfect Murder",4),("Reservoir Dogs",3)]
similar_movies = pd.DataFrame()
for movie,rating in action_movie_lover:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

similar_movies.head(10)

,"20,000 Leagues Under the Sea",2001: A Space Odyssey,A Clockwork Orange,A Close Shave,A Nightmare on Elm Street,A Perfect Murder,A Time to Kill,A View to a Kill,Addicted to Love,Alien,...,Under Siege 2: Dark Territory,Vertigo,Wag the Dog,We're No Angels,What's Eating Gilbert Grape,While You Were Sleeping,Willy Wonka & the Chocolate Factory,Young Frankenstein,Young and Innocent,eXistenZ
0,0.147369,0.164204,0.227495,0.280625,0.242453,0.065479,0.005247,0.156761,0.021344,0.194690,...,0.020482,0.245697,0.228951,-0.044686,0.134881,-0.008392,0.492851,0.199766,-0.025872,-0.030075
1,0.082148,0.051477,-0.006161,0.335224,0.167689,1.500000,0.136383,0.277148,0.096674,0.278815,...,-0.009499,0.198690,0.018339,-0.019216,0.402995,0.561462,0.088225,0.149308,0.238937,0.239455
2,0.115309,0.114797,0.192850,0.048194,0.266093,0.030020,0.080380,0.056134,0.055555,0.043270,...,0.016334,0.050311,0.260279,-0.005691,0.062180,0.028522,0.158802,0.076028,0.097320,0.008040
